In [ ]:
import sys
import requests
import urllib3
from trino.auth import JWTAuthentication
from trino.dbapi import connect
import pandas as pd


urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
KEYCLOAK_TOKEN_URL = (
    "http://localhost:30080/realms/iceberg/protocol/openid-connect/token"
)
TRINO_URL = "https://localhost:443"
CATALOG_URL = "http://localhost:8181/catalog"
WAREHOUSE = "bronze"
CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"  

In [ ]:
response = requests.post(
    KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "password",
        "client_id": "trino",
        "client_secret": "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ",
        "username": "hung",
        "password": "hung",
        "scope": "trino",
    },
)

In [6]:
access_token = response.json()["access_token"]
print(f"OK: Got token (length: {len(access_token)} chars)")

OK: Got token (length: 1283 chars)


In [17]:
conn = connect(
    host=TRINO_URL,
    auth=JWTAuthentication(access_token),
    http_scheme="https",
    verify=False,
)
cur = conn.cursor()

In [ ]:
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = 'http://localhost:8181/catalog',
        "iceberg.rest-catalog.warehouse" = 'bronze',
        "iceberg.rest-catalog.security" = 'OAUTH2',
        "iceberg.rest-catalog.oauth2.credential" = 'trino:AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "iceberg.rest-catalog.oauth2.scope" = 'lakekeeper',
        "iceberg.rest-catalog.oauth2.server-uri" = 'http://localhost:30080/realms/iceberg/protocol/openid-connect/token',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://localhost:9000',
        "fs.native-s3.enabled" = 'true'
    )
    """
)

In [29]:
conn = connect(
    host=TRINO_URL,
    auth=JWTAuthentication(access_token),
    http_scheme="https",
    verify=False,
)
cur = conn.cursor()

In [30]:
cur.execute("CREATE SCHEMA IF NOT EXISTS lakekeeper.finance")

In [ ]:
cur.execute("CREATE TABLE IF NOT EXISTS lakekeeper.finance.revenue (year INT, revenue DOUBLE)")
# cur.execute("INSERT INTO lakekeeper.finance.revenue VALUES (2026, 10355.1), (2025, 10477.2)")

In [ ]:
# Execute query and fetch all rows
cur.execute("SELECT * FROM lakekeeper.finance.revenue")
rows = cur.fetchall()

# Get column names
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display DataFrame
print(df)

   year  revenue
0  2026  10355.1
1  2026  10355.1
2  2025  10477.2
3  2025  10477.2
